In [ ]:
import hvplot.pandas
import panel as pn
import holoviews as hv

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

pn.extension()

This example demonstrates how to put widgets together to build a simple UI for exploring the distribution of sea surface temperatures, as follows:

- Declare the various widgets
- Declare a function that is decorated with the `pn.depends` to express the dependencies on the widget values
- Define a callback that pops up the bandwidth slider to control the kernel density estimate (if it has been enabled).

In [ ]:
bins = pn.widgets.IntSlider(name='Number of bins', start=5, end=25, step=5, value=10)
kde = pn.widgets.Checkbox(name='Show density estimate')
observations = pn.widgets.Checkbox(name='Show individual observations')
bandwidth = pn.widgets.FloatSlider(name='KDE Bandwidth', start=0.1, end=1)

@pn.depends(bins.param.value, kde.param.value,
            observations.param.value, bandwidth.param.value)
def get_plot(bins, kde, obs, bw):
    plot = sea_surface_temperature.hvplot.hist(bins=bins, normed=True, xlabel='Temperature (C)', padding=0.1)
    if kde:
        plot *= sea_surface_temperature.hvplot.kde().opts(bandwidth=bw)
    if obs:
        plot *= hv.Spikes(sea_surface_temperature, 'temperature').opts(
            line_width=0.1, alpha=0.1, spike_length=-0.01)
    return plot

widgets = pn.WidgetBox('## Sea surface temperatures', bins, observations, kde)

def add_bandwidth(event):
    if event.new:
        widgets.append(bandwidth)
    else:
        widgets.remove(bandwidth)

kde.param.watch(add_bandwidth, 'value')

pn.Row(widgets, get_plot).servable()